# Graphistry cuGraph bindings

Graphistry simplifies working with cuGraph. This tutorial demonstrates:

* Converting data between cpu/gpu dataframes and cuGraph
* Enriching dataframes with cuGraph algorithm results
* Visualization

An upcoming release will be adding remote GPU execution when your immediate Python environment does not have a GPU.


## Import

In [1]:
import graphistry, pandas as pd
print(graphistry.__version__)
graphistry.register(api=3, username='...', password='...')

/opt/conda/envs/rapids/lib/python3.8/site-packages/torch/cuda/__init__.py:83: UserWarning: HIP initialization: Unexpected error from hipGetDeviceCount(). Did you run some cuda functions before calling NumHipDevices() that might have already set an error? Error 101: hipErrorInvalidDevice (Triggered internally at  ../c10/hip/HIPFunctions.cpp:110.)
  return torch._C._cuda_getDeviceCount() > 0
/opt/conda/envs/rapids/lib/python3.8/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


'refs/pull/360/head'

In [22]:
df = pd.read_csv('https://raw.githubusercontent.com/graphistry/pygraphistry/master/demos/data/lesmiserables.csv')
g = graphistry.edges(df, df.columns[0], df.columns[1])
g._edges.sample(5)

,source,target,value
44,Fantine,Dahlia,4
206,Gueulemer,Valjean,1
225,Claquesous,Enjolras,1
49,Mme.Thenardier,Valjean,7
110,Gavroche,Thenardier,1


## Enrich graphs with cuGraph algorithm results and visualize the results

In [23]:
g2 = g.compute_cugraph('pagerank')
g2._nodes.sample(5)

/opt/conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(


,id,pagerank
24,Labarre,0.027145
25,Listolier,0.024138
37,Mme.Pontmercy,0.004004
56,Champtercier,0.004004
49,BaronessT,0.004004


In [24]:
g2.encode_point_color('pagerank', ['blue', 'yellow', 'red'], as_continuous=True).plot()

## Use cuGraph provided layouts
Run cuGraph's implementation of force_atlas2, inspect results, use custom parameters, and render.

Note that Graphistry's default layout algorithm is already a GPU accelerated FA2 and with additional settings.

In [25]:
g3 = g2.layout_cugraph('force_atlas2')
g3._nodes.sample(5)

/opt/conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(


,id,pagerank,x,y
63,Mme.Burgon,0.010177,100.112083,116.192375
36,Eponine,0.014482,4.713388,30.839472
38,Fantine,0.043900,-38.890110,-113.019806
67,Mme.Thenardier,0.036174,-15.617190,-7.400091
24,Champtercier,0.004004,169.118210,-157.776154


In [29]:
g3b = g2.layout_cugraph('force_atlas2', params={'lin_log_mode': True})
g3b._nodes.sample(5)

,id,pagerank,x,y
34,Mme.Burgon,0.010177,341.922455,-116.781532
56,Listolier,0.024138,312.646667,-599.275085
13,Geborand,0.004004,-17.671986,85.812553
35,Mme.Hucheloup,0.004004,325.224762,235.983505
73,Feuilly,0.006593,259.452240,416.950470


In [26]:
g3.plot()

### Convert between dataframes and cuGraph

In [30]:
G = g3.to_cugraph()

/opt/conda/envs/rapids/lib/python3.8/site-packages/cudf/core/indexed_frame.py:2271: FutureWarning: append is deprecated and will be removed in a future version. Use concat instead.
  warnings.warn(


In [31]:
G

In [36]:
G.number_of_edges()

254

In [40]:
g4 = graphistry.from_cugraph(G)

{
    'edges': g4._edges.shape,
    'nodes': g4.materialize_nodes()._nodes.shape
}

{'edges': (254, 2), 'nodes': (77, 1)}